# Preparation & EDA

You should check :
1. pattern of data
2. Missing value
3. Outliers
4. Data value

### Cleaning Data

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
path = "netflix_titles1.csv"

#path = "D/nama_folder/nama_file" if your data directory is not the same with yor ipynb file

data = pd.read_csv(path)
data.head()

In [ ]:
#shape of data
data.shape

In [ ]:
data.info()

##### check missing value

In [ ]:
data.isnull().sum()

there are 2 options when there are missing value on your data
1. change with another value
2. remove data that contains missing value

you also can call this method with manipulating data

In [ ]:
data[['director', 'cast', 'country']] = data[['director', 'cast', 'country']].fillna('null')
data['date_added'] = data['date_added'].fillna(0)
data['rating'] = data['rating'].fillna(data['listed_in'])

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
print('Type : \n', data['type'].unique())
print('-------------------------------------')
print('date_added : \n', data['date_added'].unique())
print('-------------------------------------')
print('release_year : \n', data['release_year'].unique())
print('-------------------------------------')
print('Rating : \n', data['rating'].unique())
print('-------------------------------------')
print('Genre : \n', data['listed_in'].unique())

### Cleaning kolom date_added

In [ ]:
import re

In [ ]:
test = []
for i in data['date_added']:
    if i != 0:
        find_2020 = re.findall(r'[a-zA-Z]+\s[0-9]+,\s2020', i)
        for j in find_2020:
            test.append(j)
            
unique_test = set(test)
unique_test

In [ ]:
data['date_added'] = data['date_added'].str.replace('[a-zA-Z]+\s[0-9]+,\s','')
data['date_added'] = data['date_added'].apply(lambda x: int(x) if x == x else 0)
data['date_added'].unique()

In [ ]:
data.head(2)

untuk pengaturan warna kamu dapat memilih disini https://seaborn.pydata.org/tutorial/color_palettes.html

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(data['date_added'].value_counts().index, data['date_added'].value_counts().values, palette='deep')
plt.title('Perkembangan upload data pada netflix', fontsize=17)
plt.xlabel('tahun')
plt.ylabel('jumlah')
plt.show()

Karena perbedaan yang mencolok, mari kita analisa untuk taun 2016-2020

In [ ]:
data['date_added'].head()

In [ ]:
tahun = [2016, 2017, 2018, 2019, 2020]
data_pop = data['date_added'].isin(tahun)
data2 = data[data_pop]
data2['date_added'].unique()

In [ ]:
see1 = pd.crosstab(data2['date_added'], data2['type'])
see1

In [ ]:
stacked2 = see1.stack()
stacked2

In [ ]:
type(stacked2)

In [ ]:
stacked = see1.stack().reset_index().rename(columns={0:'value'})

In [ ]:
stacked.head()

In [ ]:
type(stacked)

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(x='date_added', y='value', data=stacked, hue='type')
plt.title('Perkembangan kriteria dari tahun ke tahun')
plt.show()

### Cleaning kolom Rating

let's now google the categories and explore them

    TV-MA:This program is specifically designed to be viewed by adults and therefore may be unsuitable for children under 17.
    TV-14:This program contains some material that many parents would find unsuitable for children under 14 years of age.
    TV-PG:This program contains material that parents may find unsuitable for younger children.
    R:Under 17 requires accompanying parent or adult guardian,Parents are urged to learn more about the film before taking their young children with them.
    PG-13:Some material may be inappropriate for children under 13. Parents are urged to be cautious. Some material may be inappropriate for pre-teenagers.
    NR or UR:If a film has not been submitted for a rating or is an uncut version of a film that was submitted
    PG:Some material may not be suitable for children,May contain some material parents might not like for their young children.
    TV-Y7:This program is designed for children age 7 and above.
    TV-G:This program is suitable for all ages.
    TV-Y:Programs rated TV-Y are designed to be appropriate for children of all ages. The thematic elements portrayed in programs with this rating are specifically designed for a very young audience, including children ages 2-6.
    TV-Y7-FV:is recommended for ages 7 and older, with the unique advisory that the program contains fantasy violence.
    G:All ages admitted. Nothing that would offend parents for viewing by children.
    NC-17:No One 17 and Under Admitted. Clearly adult. Children are not admitted.
    
Akan dibedakan rating berdasarkan 3 ciri:

1. dapat ditonton segala usia (all_ages)
2. dibawah 17 tahun (under_17)
3. diatas 17 tahun (17_more)

In [ ]:
df = data2[data2['rating'] == 'Movies']
df
#kategorikan kedalam film diatas 17

In [ ]:
df1 = data2[data2['rating'] == 'Dramas']
df1
#kategorikan kedalam 17 tahun keatas

In [ ]:
df2 = data2[data2['rating'] == "Kids' TV, TV Comedies"]
df2
#dibawah 17 tahun

In [ ]:
df3 = data2[data2['rating'] == "Anime Series, International TV Shows"]
df3
# semua kalangan

In [ ]:
for j in data2.index:
    all_ = ['NR', 'UR', 'TV-G', 'G', 'Anime Series, International TV Shows']
    under_ = ['TV-Y7', 'TV-Y', 'TV-Y7-FV', "Kids' TV, TV Comedies"]
    more_ = ['TV-MA', 'TV-14', 'TV-PG', 'R', 'PG-13', 'PG', 'NC-17', 'Dramas', 'Movies']
    for i in all_:
        if data2.loc[j,'rating']== i:
            data2.loc[j,'rating']='all_ages'
    for k in under_:
        if data2.loc[j,'rating']== k:
            data2.loc[j,'rating']='under_17'
    for l in more_:
        if data2.loc[j,'rating']== l:
            data2.loc[j,'rating']='17_more'

In [ ]:
data2['rating'].unique()

In [ ]:
see2 = pd.crosstab(data2['type'], data2['rating'])
see2

In [ ]:
look2 = see2.stack().reset_index().rename(columns={0:'value'})

In [ ]:
plt.figure(figsize=(8, 8))
sns.barplot(x='type', y='value', data=look2, hue='rating', palette='Paired')
plt.title('persebaran rating dalam setiap kriteria', fontsize=15)
plt.show()

### Cleaning kolom listed_in

In [ ]:
data_movie = data2[data2['type'] == 'Movie']
data_tv = data2[data2['type'] == 'TV Show']
print(data_movie['type'].unique())
print(data_tv['type'].unique())

In [ ]:
data_movie['listed_in'].value_counts()

In [ ]:
data_tv['listed_in'].value_counts()

In [ ]:
data['listed_in'] = data.apply(lambda row: row['listed_in'].lower().split(', '), axis=1)

In [ ]:
data.head()

In [ ]:
movies = data['type'] == 'Movie'
df_movies = data[movies]
df_movies.head()

In [ ]:
list_genre =[]
for i in df_movies['listed_in']:
    for j in i:
        list_genre.append(j)
        
def unique(list_genre):
    x = np.array(list_genre)
    print(np.unique(x))
unique(list_genre)

In [ ]:
import collections

In [ ]:
freq_kata = collections.Counter(list_genre)

In [ ]:
freq_kata